In [221]:
import pandas as pd
import numpy as np
import missingno as msno
from category_encoders import TargetEncoder
from scipy.stats import gamma, kstest
from scipy import stats
import utils

In [222]:
df = pd.read_csv('hotels.csv')
print(f'Data set has {df.shape[0]} records and {df.shape[1]} parameters')


Data set has 119390 records and 32 parameters


In [223]:
date_columns =[
"arrival_date_year",
"arrival_date_month",
"arrival_date_week_number",
"arrival_date_day_of_month",
"reservation_status_date",
"stays_in_weekend_nights",
"stays_in_week_nights",
"reservation_status"
]

### Missing values
<ul>
<li>'Company' column can be eliminated since it is almost empty (95% of missing values).</li>
<li>'Agent' column has 13% of NaN</li>
<li>'previous_cancellations' and 'country' columns have less than 1% of NaN values</li>
</ul>
Thus, it is accesible to remove records comprising Nan values

In [224]:
df.isnull().mean().round(4).mul(100).sort_values(ascending=False).head(5)

company                   94.31
agent                     13.69
country                    0.41
hotel                      0.00
previous_cancellations     0.00
dtype: float64

In [225]:
df = df.drop(['company'], axis=1)
df = df.dropna()

### Combine columns related to arriving date (year, month, day) in date arriving date column
drop arring week column

In [226]:
import calendar

df['arrival_date_month'] = df['arrival_date_month'].map({v: k for k,v in enumerate(calendar.month_name)})

df.arrival_date_year = pd.to_numeric(df.arrival_date_year)
df.arrival_date_month = pd.to_numeric(df.arrival_date_month)
df.arrival_date_day_of_month = pd.to_numeric(df.arrival_date_day_of_month)

df.rename(columns = {'arrival_date_year':'year','arrival_date_month':'month','arrival_date_day_of_month':'day'}, inplace=True)
df['arrival_date'] = pd.to_datetime(df[['year','month', 'day']])
df.drop(columns=['year','month','day','arrival_date_week_number'], inplace=True)

In [227]:
df.head()

,hotel,is_canceled,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,...,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,arrival_date
3,Resort Hotel,0,13,0,1,1,0.0,0,BB,GBR,...,No Deposit,304.0,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01
4,Resort Hotel,0,14,0,2,2,0.0,0,BB,GBR,...,No Deposit,240.0,0,Transient,98.0,0,1,Check-Out,2015-07-03,2015-07-01
5,Resort Hotel,0,14,0,2,2,0.0,0,BB,GBR,...,No Deposit,240.0,0,Transient,98.0,0,1,Check-Out,2015-07-03,2015-07-01
7,Resort Hotel,0,9,0,2,2,0.0,0,FB,PRT,...,No Deposit,303.0,0,Transient,103.0,0,1,Check-Out,2015-07-03,2015-07-01
8,Resort Hotel,1,85,0,3,2,0.0,0,BB,PRT,...,No Deposit,240.0,0,Transient,82.0,0,1,Canceled,2015-05-06,2015-07-01


In [228]:
df.shape

(102894, 28)

### lead time  (Important feature)
<ul>
 <li>According to box plot we can observe huge amount of outliers </li>
 <li>with IQR technique minimum outlier value was set as 384 for whole column</li>
 <li> For canceled booking minimun outlier values is 461, and for not canceled 313 </li>
</ul>

In [229]:
utils.find_outliers_IQR(df.lead_time)

number of outliers: 2576
max outlier value: 629
min outlier value: 384


In [230]:
lead_time_upper_bound = 384
df.drop(df[df['lead_time'] > lead_time_upper_bound].index, inplace = True)

### Hotel 
#### One hot encode 'hotel' column

In [231]:
df['hotel'] =  pd.get_dummies(df.hotel, drop_first=True)
df.rename(columns={'hotel' : "resot_hotel"}, inplace=True)

### Adults and children
From disturbution plots of these columns we can observe that there is no huge variance, in another words variance considered as 'information' gained from variable. So, this 3 columns can be eliminated 

In [232]:
df.drop(columns=['adults', 'children', 'babies'], inplace=True)

### Meal 
Eliminate

In [233]:
df.drop(columns=['meal'], inplace=True)

### Country
Country column will be droped since it adds bias to the data and decrease model perfomace with origin of booking

In [234]:
df.drop(columns=['country'], inplace=True)

### market_segment and distribution_channel
Significant variable in the data 

<b>distribution_channel</b> will be one hot encoded and <b>market_segment</b> will be target encoded

In [235]:
encoder = TargetEncoder()
df['market_segment'] = encoder.fit_transform(df['market_segment'], df['is_canceled'])

In [236]:
distribution_channel = pd.get_dummies(df['distribution_channel'], drop_first=True)
df = df.drop('distribution_channel',axis = 1)
df = df.join(distribution_channel)

### is_repeated_guest
keep



### previous_cancellations and previous_bookings_not_canceled
Can be dropped 


In [237]:
df.drop(columns=['previous_cancellations','previous_bookings_not_canceled'], inplace=True)

### reserved_room_type and assigned_room_type
Tatrget encode 

In [238]:
encoder = TargetEncoder()
df['assigned_room_type'] = encoder.fit_transform(df['assigned_room_type'], df['is_canceled'])
encoder = TargetEncoder()
df['reserved_room_type'] = encoder.fit_transform(df['reserved_room_type'], df['is_canceled'])

### booking_changes
Number of changes/amendments made to the booking from the moment the booking was entered on the PMS until the moment of check-in or cancellation

Can be droped

In [239]:
df.drop(columns=['booking_changes'], inplace=True)

### deposit_type
Indication on if the customer made a deposit to guarantee the booking. This variable can assume three categories:

Since there are only 48 records with 'Refundable' type of deposit, which means deposit was made under full cost of booking,
 this records will be combined with 'Non Refund' (a deposit was made in the value of the total stay cost) and named Deposit. After that new vvariable will be one hot encoded

In [240]:
df.loc[df["deposit_type"] == "Refundable", "deposit_type"] = "deposit"
df.loc[df["deposit_type"] == "Non Refund", "deposit_type"] = "deposit"
df["deposit_type"] = np.where(df["deposit_type"] == "deposit", 1, 0)

### Agent ID

Despite of varaibility of infromation, this variable is unbalanced and top-15 agents has 75k of records out of 102k. In general there 332 agents

Drop 

In [241]:
df = df.drop(['agent'], axis=1)


### days_in_waiting_list
Number of days the booking was in the waiting list before it was confirmed to the customer

Change from continuous to categorical: 
<ul>
<li> 0-1 -day = imm</li>
<li> 1-7 -days = fast</li>
<li> 7-30 days = slow</li>
<li> 30+ = very slow</li>
</ul>

And then lable encoded

In [242]:
df['days_in_waiting_list'] = pd.cut(x=df['days_in_waiting_list'], bins=[0, 1,7, 30,900],
                     labels=['imm', 'fast', 'slow',
                             'very'])

In [243]:
encoder = TargetEncoder()
df['days_in_waiting_list'] = encoder.fit_transform(df['days_in_waiting_list'], df['is_canceled'])

### customer_type
#### Type of booking, assuming one of four categories:
<ul>
<li> Contract - when the booking has an allotment or other type of contract associated to it</li>
<li> Group – when the booking is associated to a group</li>
<li> Transient – when the booking is not part of a group or contract, and is not associated to other transient booking</li>
<li> Transient-party – when the booking is transient, but is associated to at least other transient booking</li>
</ul>


Target Encoded

In [244]:
encoder = TargetEncoder()
df['customer_type'] = encoder.fit_transform(df['customer_type'], df['is_canceled'])

### adr
Average Daily Rate as defined by dividing the sum of all lodging transactions by the total number of staying nights

Dealing with outliers

In [245]:
Q1 = df.adr.quantile(0.25)
Q3 = df.adr.quantile(0.75)
IQR = Q3 - Q1
df.drop(df[df['adr'] < (Q1 - 1.5 * IQR)].index, inplace = True)
df.drop(df[df['adr'] > (Q3 + 1.5 * IQR)].index, inplace = True)


### required_car_parking_spaces
Number of car parking spaces required by the customer

Keep

In [246]:
df.drop(columns=['required_car_parking_spaces'], inplace = True)


### total_of_special_requests
Number of special requests made by the customer (e.g. twin bed or high floor)

Target encoded

In [247]:
encoder = TargetEncoder()
df['total_of_special_requests'] = encoder.fit_transform(df['total_of_special_requests'], df['is_canceled'])

### reservation status column
Needs to be droped since highly correlated with target variable

In [249]:
df.drop(columns=['reservation_status'], inplace = True)


In [251]:
df.drop(columns=['reservation_status_date','arrival_date'], inplace = True)


In [254]:
df.to_csv('hotels_processed.csv',index=False)